In [32]:
!pip install openai 


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
# Importing Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openai
import json
import os
from tqdm import tqdm

In [ ]:
def analyze_text(text):
    prompt = f"""
        Analyze the following dialogue and extract persuasion-related factors. 
        For each factor listed below, provide a single-word answer ("Yes" or "No") in JSON format. 
        For factors like argument type and concreteness, provide the appropriate label 
        (e.g., "Logical", "Emotional", or "Concrete", "Abstract").

        ### Factors:
        1. Framing (Yes/No)
        2. Priming (Yes/No)
        3. Repetition (Yes/No)
        4. Narrative/Story (Yes/No)
        5. Reciprocity (Yes/No)
        6. Commitment and Consistency (Yes/No)
        7. Social Proof (Yes/No)
        8. Scarcity (Yes/No)
        9. Authority (Yes/No)
        10. Liking (Yes/No)
        11. Argument Type ("Logical" or "Emotional")
        12. Hedging and Politeness (Yes/No)
        13. Pronoun Use ("we", "us") (Yes/No)
        14. Active Tense (Yes/No)
        15. Concreteness ("Concrete" or "Abstract")
        16. Linguistic Fluency (Yes/No)
        17. Linguistic Mirroring (Yes/No)
        18. Credibility Markers (Yes/No)
        19. Certainty Markers (Yes/No)
        20. Rhetorical Questions (Yes/No)

        ### Dialogue:
        {text}
        """
    open_api_key = os.environ.get("OPENAI_API_KEY")
    client = openai.OpenAI(api_key=open_api_key)
    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that analyzes the dialogue \
                                                for persuasive elements. Just answer in one word."},
                {"role": "user", "content": f"Analyze the following text:\n\n{prompt}"}
            ]
        )
    output = response.choices[0].message.content.strip()
    cleaned_string = output.strip().strip("'").replace('```json\n', '').replace('```', '').replace('\n', '')

    # Convert the cleaned string into a Python dictionary (JSON object)
    json_object = json.loads(cleaned_string)

    return json_object


In [35]:
text = """Persuader: Hey, how are you doing, have you ever heard of a charity called Save the Children?
Persuadee: No I havent.
Persuadee: What is it about?
Persuader: It's an international organization that promotes children's rights, provides relief and helps support children in developing countries, does that sound like something you'd get behind?
Persuadee: No thank you.
Persuader: May I ask why?
Persuader: Your help could really make a difference in a child's life, think of a child who didn't have enough to eat, just a small bit of help may have saved them, are you sure you're not interested?
Persuadee: I am already making a difference in many children lives.
Persuader: That's great, it sounds like it's an issue you really care about then, why not put some of that effort into a charity that's doing really great work like Save the Children?
Persuadee: I have donated 15 years of my life to charity
Persuader: I'm not asking you to donate years of your life, just a couple of cents if possible, is that something you could possibly do tonight?
Persuadee: No thank you
Persuader: You're probably on a computer right now, so you probably have quite a bit of money yourself, don't you think you could spare maybe 25 cents for some needy children around the world?
Persuadee: No thank you
Persuader: Do you have children yourself?
Persuader: What if your children were in a position these kids are in, wouldn't you want someone to help them?
Persuadee: Where are those childrens parent?
Persuader: If only they had parental support, but in war-torn countries like Syria their parents are being killed in wars or are fighting for their country, they have no help and that's why they need your support, please find it in your heart to just give a little bit tonight, that's all I'm asking.
Persuadee: My small change won't do a thing for those kids.
Persuadee: What they need is to be extracted from those countries so that they can have a better life not 0.25
Persuader: Well with our current president it's not likely they're going to be brought the US, so they need any help they can get, and 25 cents might not seem like a lot, but if 10000 people donated 25 cents then the impact will be huge.
Persuader: I understand it may not seem like a lot, but it can really make a difference
Persuader: You can donate some or all of your payment from this task to Save the Children, i'd really appreciate it if you did, but I understand if you still don't want to
Persuader: What do you say?
Persuadee: If it makes you happy, go ahead and take .50
Persuader: Oh thank you so much, you have no idea how much that .50 will mean to those children, thank you very much and have a wonderful night you blessed soul.
Persuadee: No problem have a good night
"""

In [36]:
tmp = analyze_text(text)

In [37]:
#### Data

import pandas as pd

# Load the persuasion data under the data folder
dialog_data = pd.read_excel('data\\persuasion_for_good\\300_dialog.xlsx')

result = dialog_data.groupby('B2').apply(
    lambda x: f'{os.linesep}'.join(f"{'Persuadee' if row['B4'] == 1 else 'Persuader'}: {row['Unit']}" for _, row in x.iterrows())
).reset_index(name='Combined_Text')

result.head()

,B2,Combined_Text
0,20180717-200206_41_live,"Persuader: Hey, how are you doing, have you ev..."
1,20180719-120436_413_live,Persuader: Hi there\r\nPersuadee: Hello\r\nPer...
2,20180719-122534_38_live,Persuader: It would be really helpful if you d...
3,20180719-165941_192_live,"Persuader: Hello\r\nPersuadee: Yea, hello.\r\n..."
4,20180719-175233_833_live,Persuader: Hey there\r\nPersuadee: Hey how's i...


In [ ]:

# result['Analysis'] = result['Combined_Text'].apply(analyze_text)
# Modify the code to do in a iterative fashion

# Create an empty list to store the results
# results = []
# i = 0
# # Iterate over each dialogue in the dataset
# for dialog in tqdm(result['Combined_Text']):
#     # Analyze the dialogue
#     analysis = analyze_text(dialog)
#     # Append the results to the list
#     results.append(analysis)
#     i += 1
#     if i % 50 ==0:
#         break

 16%|█▋        | 49/300 [03:24<17:29,  4.18s/it]


In [51]:
dummy = []
i = 0
# Iterate over each dialogue in the dataset
for dialog in tqdm(result['Combined_Text']):
    # Analyze the dialogue
    analysis = analyze_text(dialog)
    # Append the results to the list
    dummy.append(analysis)

100%|██████████| 300/300 [17:12<00:00,  3.44s/it]


Framing                         3
Priming                         3
Repetition                      3
Narrative/Story                 3
Reciprocity                     3
Commitment and Consistency      3
Social Proof                    3
Scarcity                        3
Authority                       3
Liking                          3
Argument Type                   3
Hedging and Politeness          3
Pronoun Use                     3
Active Tense                    3
Concreteness                    3
Linguistic Fluency              3
Linguistic Mirroring            3
Credibility Markers             3
Certainty Markers               3
Rhetorical Questions            3
1                             297
2                             297
3                             297
4                             297
5                             297
6                             297
7                             297
8                             297
9                             297
10            

In [53]:
# Convert the dictionary into columns and add to the results dataframe
res_df = pd.DataFrame(dummy)
res_df.to_csv('data\\persuasion_for_good\\persuasion_analysis.csv', index=False)

In [62]:
result_df = pd.concat([result, res_df], axis=1)

In [65]:
result_df[result_df['11'].notna()]

,B2,Combined_Text,Framing,Priming,Repetition,Narrative/Story,Reciprocity,Commitment and Consistency,Social Proof,Scarcity,...,11,12,13,14,15,16,17,18,19,20
76,20180808-040435_931_live,Persuader: Good Morning!\r\nPersuader: I'm hop...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Emotional,Yes,Yes,Yes,Concrete,Yes,No,Yes,No,Yes
190,20180826-045748_926_live,Persuader: hello haw are you today\r\nPersuade...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Emotional,Yes,Yes,No,Concrete,Yes,No,No,Yes,Yes
216,20180826-173836_197_live,"Persuader: Hi, How are you today?\r\nPersuadee...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Emotional,No,Yes,Yes,Concrete,Yes,No,Yes,Yes,Yes


In [40]:
# # Create DataFrames
# dataset["metrics"] = all_metrics

# # Save to CSV files
# dataset.to_csv('data/metrics.csv', index=False)

# print("Processing complete. Results saved to CSV files.")

In [ ]:
# # Load the dataset
# dataset = pd.read_csv("data/persuasion_data.csv")

# # Create empty lists to store the results
# all_metrics = []
# all_claims = []
# all_arguments = []

# # Process each example in the dataset
# for i, row in tqdm(dataset.iterrows()):
#     claim = row['claim']
#     argument = row['argument']
    
#     try:
#         metrics = analyze_text("This is the claim \n" + claim + "\n" + "The argument for claim is \n" + argument)
#         all_metrics.append(metrics)
#         # print(f"Processed claim: {claim[:50]}...")
#     except Exception as e:
#         all_metrics.append("")
#         # print(f"An error occurred: {str(e)}")

#     # if i == 5:
#     #     break

# metrics_df = pd.DataFrame(all_metrics)
# metrics_df.to_csv("data/base_metrics.csv", index=False)

5it [00:21,  4.34s/it]


KeyboardInterrupt: 